In [2]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns


#model selection
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Add,Dropout, Dense, Activation, ZeroPadding2D, \
BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
import PIL
import PIL.Image

import glob   
import shutil


In [3]:
data_dir = os.curdir + "/Flowers"
# Training data dir
training_dir = os.curdir + '/Train'

# Test data dir
testing_dir = os.curdir + '/Test'

# Ratio of training and testing data
train_test_ratio = 0.8 

def split_dataset_into_test_and_train_sets(all_data_dir = data_dir, training_data_dir = training_dir, \
                                           testing_data_dir=testing_dir, train_test_ratio = 0.8):
    # Recreate testing and training directories
    
    if not os.path.exists(training_data_dir):
            os.mkdir(training_data_dir)

    if not os.path.exists(testing_data_dir):
            os.mkdir(testing_data_dir)               
    
    num_training_files = 0
    num_testing_files = 0


    for subdir, dirs, files in os.walk(all_data_dir):
        
        category_name = os.path.basename(subdir)
        
        # print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
              continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name
        
        # creating subdir for each sub category
        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)   

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)
            
        file_list = glob.glob(os.path.join(subdir,'*.jpg'))

        #print(os.path.join(all_data_dir, subdir))
        print(str(category_name) + ' has ' + str(len(files)) + ' images') 
        random_set = np.random.permutation((file_list))
        # copy percentage of data from each category to train and test directory
        train_list = random_set[:round(len(random_set)*(train_test_ratio))] 
        test_list = random_set[-round(len(random_set)*(1-train_test_ratio)):]

  

        for lists in train_list : 
            shutil.copy(lists, training_data_dir + '/' + category_name + '/' )
            num_training_files += 1
  
        for lists in test_list : 
            shutil.copy(lists, testing_data_dir + '/' + category_name + '/' )
            num_testing_files += 1
  

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [4]:
split_dataset_into_test_and_train_sets()

Babi has 931 images
Calimerio has 353 images
Chrysanthemum has 696 images
Hydrangeas has 518 images
Lisianthus has 969 images
Pingpong has 360 images
Rosy has 171 images
Tana has 623 images
Processed 3696 training files.
Processed 925 testing files.


In [5]:
# Defining the imagedatagenerator for train and test image for pre-processing
# We don't give horizonal_flip or other preprocessing for validation data generator

image_size = 224
batch_size = 64

train_data_gen = ImageDataGenerator(
    rescale = 1./255, #normalization
    rotation_range = 40,
    width_shift_range = 0.4,
    height_shift_range = 0.4,
    shear_range = 0.2,
    zoom_range = 0.1,
    fill_mode = 'nearest',
    horizontal_flip = True)
valid_data_gen = ImageDataGenerator(rescale = 1./255)
train_generator = train_data_gen.flow_from_directory(training_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_data_gen.flow_from_directory(testing_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')

Found 3696 images belonging to 8 classes.
Found 925 images belonging to 8 classes.


In [16]:
def create_cnn_model(dropout_rate, learning_rate):
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (224,224,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
    model.add(Dropout(dropout_rate))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
    model.add(Dropout(dropout_rate))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Flatten())

    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(dropout_rate))

    model.add(Dense(8, activation = "softmax"))
    # Compile model
    model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy',metrics=['accuracy'])  
    return model

In [ ]:
model = KerasClassifier(build_fn=create_cnn_model, verbose=1, epochs)
# define parameters and values for grid search 
param_grid = {
    'dropout_rate': [0.2, 0.4, 0.6,
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train, to_categorical(y_train))

In [11]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Dropout(0.4))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())

model.add(Dense(512, activation = 'relu')), 

model.add(Dense(8, activation = "softmax"))

In [12]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 224, 224, 32)      2432      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 dropout_6 (Dropout)         (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 56, 56, 96)       

In [13]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=100,
    verbose=1)

Epoch 1/100
57/57 [==============================] - 132s 2s/step - loss: 2.1016 - accuracy: 0.2115 - val_loss: 2.0189 - val_accuracy: 0.2600
Epoch 2/100
57/57 [==============================] - 120s 2s/step - loss: 1.9357 - accuracy: 0.2428 - val_loss: 1.9366 - val_accuracy: 0.2478
Epoch 3/100
57/57 [==============================] - 122s 2s/step - loss: 1.8761 - accuracy: 0.2830 - val_loss: 1.8479 - val_accuracy: 0.3214
Epoch 4/100
57/57 [==============================] - 116s 2s/step - loss: 1.8141 - accuracy: 0.3111 - val_loss: 1.7913 - val_accuracy: 0.3728
Epoch 5/100
57/57 [==============================] - 120s 2s/step - loss: 1.7538 - accuracy: 0.3527 - val_loss: 1.7033 - val_accuracy: 0.4208
Epoch 6/100
57/57 [==============================] - 120s 2s/step - loss: 1.6646 - accuracy: 0.4023 - val_loss: 1.7132 - val_accuracy: 0.3705
Epoch 7/100
57/57 [==============================] - 120s 2s/step - loss: 1.6546 - accuracy: 0.4034 - val_loss: 1.5947 - val_accuracy: 0.4621
Epoch 

57/57 [==============================] - 119s 2s/step - loss: 0.7756 - accuracy: 0.7192 - val_loss: 1.0555 - val_accuracy: 0.6283
Epoch 59/100
57/57 [==============================] - 118s 2s/step - loss: 0.7976 - accuracy: 0.7104 - val_loss: 1.0145 - val_accuracy: 0.6607
Epoch 60/100
57/57 [==============================] - 119s 2s/step - loss: 0.7767 - accuracy: 0.7247 - val_loss: 1.0210 - val_accuracy: 0.6306
Epoch 61/100
57/57 [==============================] - 118s 2s/step - loss: 0.7615 - accuracy: 0.7263 - val_loss: 0.9625 - val_accuracy: 0.6797
Epoch 62/100
57/57 [==============================] - 119s 2s/step - loss: 0.7414 - accuracy: 0.7393 - val_loss: 0.9842 - val_accuracy: 0.6618
Epoch 63/100
57/57 [==============================] - 119s 2s/step - loss: 0.7357 - accuracy: 0.7343 - val_loss: 0.9393 - val_accuracy: 0.6663
Epoch 64/100
57/57 [==============================] - 119s 2s/step - loss: 0.7621 - accuracy: 0.7249 - val_loss: 0.9988 - val_accuracy: 0.6462
Epoch 65/100

In [14]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Dropout(0.2))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())

model.add(Dense(512, activation = 'relu')), 

model.add(Dense(8, activation = "softmax"))

In [15]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 224, 224, 32)      2432      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 dropout_9 (Dropout)         (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 56, 56, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 56, 56, 96)       

In [16]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=100,
    verbose=1)

Epoch 1/100
57/57 [==============================] - 119s 2s/step - loss: 2.0564 - accuracy: 0.1867 - val_loss: 1.9841 - val_accuracy: 0.2121
Epoch 2/100
57/57 [==============================] - 118s 2s/step - loss: 1.9771 - accuracy: 0.2073 - val_loss: 1.9697 - val_accuracy: 0.1998
Epoch 3/100
57/57 [==============================] - 119s 2s/step - loss: 1.9606 - accuracy: 0.2148 - val_loss: 1.9339 - val_accuracy: 0.2165
Epoch 4/100
57/57 [==============================] - 119s 2s/step - loss: 1.9335 - accuracy: 0.2296 - val_loss: 1.8935 - val_accuracy: 0.3025
Epoch 5/100
57/57 [==============================] - 119s 2s/step - loss: 1.8780 - accuracy: 0.2789 - val_loss: 1.7823 - val_accuracy: 0.3326
Epoch 6/100
57/57 [==============================] - 118s 2s/step - loss: 1.7868 - accuracy: 0.3241 - val_loss: 1.6883 - val_accuracy: 0.3761
Epoch 7/100
57/57 [==============================] - 118s 2s/step - loss: 1.6504 - accuracy: 0.4014 - val_loss: 1.5143 - val_accuracy: 0.4799
Epoch 

57/57 [==============================] - 119s 2s/step - loss: 0.7786 - accuracy: 0.7208 - val_loss: 0.8617 - val_accuracy: 0.6975
Epoch 59/100
57/57 [==============================] - 119s 2s/step - loss: 0.7369 - accuracy: 0.7368 - val_loss: 0.8856 - val_accuracy: 0.6975
Epoch 60/100
57/57 [==============================] - 119s 2s/step - loss: 0.7348 - accuracy: 0.7423 - val_loss: 0.9368 - val_accuracy: 0.6529
Epoch 61/100
57/57 [==============================] - 119s 2s/step - loss: 0.7620 - accuracy: 0.7148 - val_loss: 0.9032 - val_accuracy: 0.6908
Epoch 62/100
57/57 [==============================] - 119s 2s/step - loss: 0.7134 - accuracy: 0.7406 - val_loss: 0.8600 - val_accuracy: 0.7031
Epoch 63/100
57/57 [==============================] - 119s 2s/step - loss: 0.7060 - accuracy: 0.7506 - val_loss: 0.8274 - val_accuracy: 0.6987
Epoch 64/100
57/57 [==============================] - 119s 2s/step - loss: 0.7363 - accuracy: 0.7360 - val_loss: 0.8529 - val_accuracy: 0.7165
Epoch 65/100

In [17]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Dropout(0.6))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Dropout(0.6))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Dropout(0.6))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())

model.add(Dense(512, activation = 'relu')), 

model.add(Dense(8, activation = "softmax"))

In [18]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 224, 224, 32)      2432      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 dropout_12 (Dropout)        (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 56, 56, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 56, 56, 96)       

In [19]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=100,
    verbose=1)

Epoch 1/100
57/57 [==============================] - 121s 2s/step - loss: 2.3742 - accuracy: 0.1963 - val_loss: 2.0299 - val_accuracy: 0.2054
Epoch 2/100
57/57 [==============================] - 120s 2s/step - loss: 1.9791 - accuracy: 0.2032 - val_loss: 2.0264 - val_accuracy: 0.2109
Epoch 3/100
57/57 [==============================] - 120s 2s/step - loss: 1.9772 - accuracy: 0.2093 - val_loss: 2.0042 - val_accuracy: 0.2076
Epoch 4/100
57/57 [==============================] - 120s 2s/step - loss: 1.9642 - accuracy: 0.2065 - val_loss: 1.9783 - val_accuracy: 0.2020
Epoch 5/100
57/57 [==============================] - 120s 2s/step - loss: 1.9362 - accuracy: 0.2137 - val_loss: 1.9498 - val_accuracy: 0.2578
Epoch 6/100
57/57 [==============================] - 120s 2s/step - loss: 1.9144 - accuracy: 0.2340 - val_loss: 1.9491 - val_accuracy: 0.2176
Epoch 7/100
57/57 [==============================] - 120s 2s/step - loss: 1.9089 - accuracy: 0.2373 - val_loss: 1.9478 - val_accuracy: 0.2020
Epoch 

57/57 [==============================] - 121s 2s/step - loss: 0.9364 - accuracy: 0.6619 - val_loss: 1.3151 - val_accuracy: 0.5167
Epoch 59/100
57/57 [==============================] - 121s 2s/step - loss: 0.9278 - accuracy: 0.6677 - val_loss: 1.4696 - val_accuracy: 0.4699
Epoch 60/100
57/57 [==============================] - 121s 2s/step - loss: 0.9676 - accuracy: 0.6520 - val_loss: 1.3266 - val_accuracy: 0.5190
Epoch 61/100
57/57 [==============================] - 121s 2s/step - loss: 0.9225 - accuracy: 0.6743 - val_loss: 1.3186 - val_accuracy: 0.5402
Epoch 62/100
57/57 [==============================] - 121s 2s/step - loss: 0.9256 - accuracy: 0.6735 - val_loss: 1.3359 - val_accuracy: 0.5022
Epoch 63/100
57/57 [==============================] - 121s 2s/step - loss: 0.9262 - accuracy: 0.6718 - val_loss: 1.4525 - val_accuracy: 0.4676
Epoch 64/100
57/57 [==============================] - 121s 2s/step - loss: 0.9219 - accuracy: 0.6696 - val_loss: 1.2462 - val_accuracy: 0.5491
Epoch 65/100